# 블로그 컨텐츠 수집(크롤링)

## #01. 준비과정
### [1] 패키지 참조


In [1]:
import datetime as dt
import requests
import os
import time
from bs4 import BeautifulSoup
import markdownify as mf
from tqdm.notebook import tqdm

## #02. 데이터 요청하기
### [1] 세션 생성

In [2]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

### [2] 전체 글 목록 가져오기

#### (1) 함수정의


In [3]:
def getContents(session, url):
    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)


    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)

    post = soup.select('.post')
    mydata =[]
    for i, v in enumerate(post):
        # print(v)

        # 하나의 글 안에서 제목 영역을 찾는다.
        entryTitle = v.select(".entry-title a")
        # print(entryTitle)

        # 추출된 요소가 각 게시글 안에서 하나만 존재하므로 `0`번째 원소에 직접 접근한다.
        title = entryTitle[0].text.strip()
        # print(title)

        # 클릭시 이동할 페이지의 주소
        if 'href' in entryTitle[0].attrs:
            href = entryTitle[0].attrs['href']

        # 같은 페이지 내에서 주소 이동할 경우 도메인 생략가능
            
        # 수집된 주소에 도메인이 없다면 덧붙여준다
        if url not in href:
            href = url+href

        else:
            href = None
        # print(href)

        # 작성일
        published = v.select(".published")
        # print(published)
        datetime = published[0].attrs['datetime']
        # print(datetime)
        
        # 요약글
        entryContent = v.select(".entry-content p")
        # print(entryContent)

        # 마지막의 `more` 버튼은 제거한다.
        # entryContent = entryContent[:-1] # python적 접근
        # print(entryContent)

        entryContent = v.select(".entry-content p:not(.read-more)") # CSS적 접근
        # print(entryContent)

        text = ''
        for j, e in enumerate(entryContent): # 2개 이상인 경우도 있으므로 반복문
            text += " " +e.text.strip() if j>0 else e.text.strip()
        # print(text)

        # 태그
        tagLinks = v.select(".tag-links a")
        # print(tagLinks)

        
        for j,e in enumerate(tagLinks):
            tagLinks[j] = e.text.strip()
        
        tags = ','.join(tagLinks)
        
        # 수집된 정보를 하나의 딕셔너리로 묶는다 => 처리방법은 다르게해도 상관 없음
        mydict = {
            "title":title,
            "href":href,
            "datetime":datetime,
            "text":text,
            "tags":tags
        }
        mydata.append(mydict)
    return mydata


#### (2) 전체 글 목록 수집
##### 접근 URL의 패턴과 최대 페이지 수 정의

In [4]:
url = "https://blog.hossam.kr"
urlFmt = "%s/blog/page{pagenumber}" %url
maxPage = 22


##### 글 목록 수집

In [5]:
blogPosts = []
for i in range(1,maxPage+1):
    targetUrl = urlFmt.format(pagenumber = i) if i>1 else url
    blogPosts += getContents(session,targetUrl)

print("수집된 전체 게시글 수 :",len(blogPosts))

수집된 전체 게시글 수 : 108


### [4] 게시글 별로 상세 내용 가져오기

#### [1] 함수 정의하기

In [6]:
def getContentBody(session, url):
    domain = "https://blog.hossam.kr"
    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    # 게시글 영역 추출
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)
    content = soup.select(".entry-content")[0]
    # 문자열로 변환
    body = str(content)
    # 이미지부분 조정
    
    rebody = body.replace('src="/',f'src="{domain}/')
    # 마크다운으로 변경
    md = mf.markdownify(rebody)
    return md

#### (3) 활용하기

In [7]:
dirname = dt.datetime.now().strftime('블로그_글_수집_%y%m%d_%H%M%S')

if not os.path.exists(dirname):
    os.mkdir(dirname)
    
progress = tqdm(total = len(blogPosts))

for i in blogPosts:
    # print(i)
    md = getContentBody(session, i['href'])

    fname = f"{dirname}/{i['datetime']}_{i['title']}.md"
    fname = fname.replace('-','').replace(' ','_')
    
    with open(fname, 'w', encoding='utf-8') as f:
        f.write(md)
    
    print(f"{fname}(이)가 저장되었습니다.")
    progress.update()
    time.sleep(0.1)

  0%|          | 0/108 [00:00<?, ?it/s]

블로그_글_수집_231213_161309/20230821_[Toy_Project]_서버_가동_설정_수정.md(이)가 저장되었습니다.
블로그_글_수집_231213_161309/20230820_[Toy_Project]_Next.js와_Express_병합.md(이)가 저장되었습니다.
블로그_글_수집_231213_161309/20230817_[Toy_Project]_토이_프로젝트_시작.md(이)가 저장되었습니다.
블로그_글_수집_231213_161309/20230713_[Mac]_OpenJDK_설치하기.md(이)가 저장되었습니다.
블로그_글_수집_231213_161309/20230712_[Mac]_개발환경_구성_필수_요소.md(이)가 저장되었습니다.
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
블로그_글_수집_231213_161309/20221231_[R]_오픈API_연동__카카오_OpenAPI_연동.md(이)가 저장되었습니다.
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
블로그_글_수집_231213_161309/20221230_[R]_오픈API_연동__영화_진흥_위원회_박스_오피스_데이터_활용.md(이)가 저장되었습니다.
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
블로그_글_수집_231213_161309/20221229_[R]_JSON의_이해.md(이)가 저장되었습니다.
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
블로그_글_수집_231213_161309/20221228_[R]_시계열_분석.md(이)가 저장되었습니다.
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
블로그_글_수집_231213_161309/20221227_[R]_회귀분석.md(이)가 저장되었습니다.
접속에 실패했습니다.
[404 Error] Not Found 에러가 발생함
블로그_글_수집_231213_161309/20221226_[R]_상관분석.md(